In [375]:
import http.client

import pymysql
import json
import unidecode
import math 
import re

In [358]:
def connect_db():
  conn = pymysql.connect(
    host="mysql-36978b5d-aurelionadjabarreto-0fc2.aivencloud.com",
    port = 18599,
    user="luciano",
    password="", # to-do fill
    db="resolucoesuefsdb",
    cursorclass=pymysql.cursors.DictCursor,
    connect_timeout=20000
  )
  return conn  

''' 
  Lowercase, accents, split and then remove stopwords
'''
# TODO: impl. regex to remove special characters.
def preprocess(queryString):
  stopwords = ["a", "à", "ao", "aos", "aquela", "aquelas", "aquele", "aqueles", "aquilo", "as", "às", "até", "com", "como", "da", "das", "de", "dela", "delas", "dele", "deles", "depois", "do", "dos", "e", "é", "ela", "elas", "ele", "eles", "em", "entre", "era", "eram", "éramos", "essa", "essas", "esse", "esses", "esta", "está", "estamos", "estão", "estar", "estas", "estava", "estavam", "estávamos", "este", "esteja", "estejam", "estejamos", "estes", "esteve", "estive", "estivemos", "estiver", "estivera", "estiveram", "estivéramos", "estiverem", "estivermos", "estivesse", "estivessem", "estivéssemos", "estou", "eu", "foi", "fomos", "for", "fora", "foram", "fôramos", "forem", "formos", "fosse", "fossem", "fôssemos", "fui", "há", "haja", "hajam", "hajamos", "hão", "havemos", "haver", "hei", "houve", "houvemos", "houver", "houvera", "houverá", "houveram", "houvéramos", "houverão", "houverei", "houverem", "houveremos", "houveria", "houveriam", "houveríamos", "houvermos", "houvesse", "houvessem", "houvéssemos", "isso", "isto", "já", "lhe", "lhes", "mais", "mas", "me", "mesmo", "meu", "meus", "minha", "minhas", "muito", "na", "não", "nas", "nem", "no", "nos", "nós", "nossa", "nossas", "nosso", "nossos", "num", "numa", "o", "os", "ou", "para", "pela", "pelas", "pelo", "pelos", "por", "qual", "quando", "que", "quem", "são", "se", "seja", "sejam", "sejamos", "sem", "ser", "será", "serão", "serei", "seremos", "seria", "seriam", "seríamos", "seu", "seus", "só", "somos", "sou", "sua", "suas", "também", "te", "tem", "tém", "temos", "tenha", "tenham", "tenhamos", "tenho", "terá", "terão", "terei", "teremos", "teria", "teriam", "teríamos", "teu", "teus", "teve", "tinha", "tinham", "tínhamos", "tive", "tivemos", "tiver", "tivera", "tiveram", "tivéramos", "tiverem", "tivermos", "tivesse", "tivessem", "tivéssemos", "tu", "tua", "tuas", "um", "uma", "você", "vocês", "vos" ]
  #keywords = unidecode.unidecode(queryString['palavras_chave'].lower(), 'utf-8').split(' ') # returns to lower
  # To-do remove line below that return tokens in regular case.
  keywords = unidecode.unidecode(queryString, 'utf-8').split(' ')
  return [word for word in keywords if word not in stopwords]

# Find the documents that contains one of more occurences of a term 
def find_occurrences(connection, term):
  cursor = connection.cursor()  
  cursor.execute("SELECT id FROM termos WHERE term=(%s)", term)
  rows = cursor.fetchall()
  if len(rows) > 0:
    id = rows[0]['id']
    cursor.execute("SELECT id_resolucao,frequencia FROM documentos WHERE id_termo=(%s)", str(id))
    rows = cursor.fetchall()
  else:
    rows = []
  return rows

def documents(connection):
  cursor = connection.cursor()  
  cursor.execute("SELECT * FROM resolucoes")
  rows = cursor.fetchall()
  return rows

# Search Algorithm:
def search(connection,query):
  keywords = preprocess(query)
  docs = documents(connection)
  N = len(docs)
  scores = {}
  for doc in docs:
    document_id = doc['id']
    w_d = doc['wd']
    s_d = 0
    w_qt = 0
    for term in keywords:
      id_list = find_occurrences(connection, term)
      f_t = len(id_list)
      id_list = dict((x,y) for x, y in [(e['id_resolucao'], e['frequencia'])for e in id_list])
      w_dt = 1
      if f_t > 0: # division by zero 
        w_qt += math.log(1 + (N/f_t)) 
        try:
          f_dt = id_list[document_id] # frequency of term in document d
        except KeyError:
          f_dt = 1
        w_dt = 1 + math.log(f_dt) # weight of term in document d
      s_d += w_qt * w_dt # score of document d
    s_d /= w_d
    scores[document_id] = {'score':s_d}
  scores = dict(sorted(scores.items(), key=lambda item: item[1]['score'], reverse=True))
  return scores, docs

def filter_documents_by_year(year, scores, documents):
  filtered_documents = [document_list[key] for key in scores.keys() if year in document_list[key]['ano']]
  arr = []
  for doc in filtered_documents:
    arr.append({
      "ano": doc['ano'],
      "orgao": "orgão responsavel... (Faltou no banco)",
      "descricao": re.sub('[^a-zA-Z0-9 \n\ \ .]', ' ', unidecode.unidecode(doc['texto'], 'utf-8').strip().replace('\n',' ')),
      "numero": doc['numero'],
      "link": doc['link']
    })
  return arr

# Search Algorithm:
def search(connection,query):
  keywords = preprocess(query)
  docs = documents(connection)
  N = len(docs)
  scores = {}
  for doc in docs:
    document_id = doc['id']
    w_d = doc['wd']
    s_d = 0
    w_qt = 0
    for term in keywords:
      id_list = find_occurrences(connection, term)
      f_t = len(id_list)
      id_list = dict((x,y) for x, y in [(e['id_resolucao'], e['frequencia'])for e in id_list])
      w_dt = 1
      if f_t > 0: # division by zero 
        w_qt += math.log(1 + (N/f_t)) 
        try:
          f_dt = id_list[document_id] # frequency of term in document d
        except KeyError:
          f_dt = 1
        w_dt = 1 + math.log(f_dt) # weight of term in document d
      s_d += w_qt * w_dt # score of document d
    s_d /= w_d
    scores[document_id] = {'score':s_d}
  scores = dict(sorted(scores.items(), key=lambda item: item[1]['score'], reverse=True))
  return scores, docs

def filter_documents_by_year(year, scores, documents):
  filtered_documents = [document_list[key] for key in scores.keys() if year in document_list[key]['ano']]
  arr = []
  for doc in filtered_documents:
    arr.append({
      "ano": doc['ano'],
      "orgao": "orgão responsavel... (Faltou no banco)",
      "descricao": re.sub('[^a-zA-Z0-9 \n\ \ .]', ' ', unidecode.unidecode(doc['texto'], 'utf-8').strip()),
      "numero": doc['numero'],
      "link": doc['link']
    })
  return arr

In [385]:
# Termos de busca
query = {
  "palavras_chave":" PROJETO DE EXTENSAO ELABORAR COM FOUCAULT",
  "data_inicio":"",
  "data_final":"2017"
}

def retrieve(num_documents, query):
  connection = connect_db()
  scores, document_list = search(connection, query['palavras_chave'])
  document_list = dict([(document.pop('id'), document) for document in document_list])
  year = query['data_final']
  filtered_documents = filter_documents_by_year(year, scores, document_list)
  return json.dumps(filtered_documents[0 : num_documents])
filtered_documents = retrieve(10, query)


In [386]:
filtered_documents = retrieve(10, query)
filtered_documents

'[{"ano": "2017", "orgao": "org\\u00e3o responsavel... (Faltou no banco)", "descricao": "REITORIA UEFS\\nPUBLICADO D.O.E.\\nEm  15 11 2017\\n\\n   \\n\\n etrienniets  \\n oF oe\\n\\n \\n\\n \\n\\nUNIVERSIDADE ESTADUAL DE FEIRA DE SANTANA\\nAutorizada pelo Decreto Federal ndeg 77.496 de 27 04 76\\nReconhecida pela Portaria Ministerial ndeg 874 86 de 19 12 86\\nRecredenciada pelo Decreto ndeg 9.271 de 14 12 2004\\nRecredenciada pelo Decreto ndeg 17.228 de 25 11 2016\\n\\nRESOLUCAO CONSEPE 120 2017\\n\\nO Reitor da Universidade Estadual de Feira de Santana e Presidente do\\nCONSELHO SUPERIOR DE ENSINO  PESQUISA E EXTENSAO  no uso de suas\\natribuigdes \\n\\nRESOLVE \\n\\nArtigo 1deg     Aprovar o PROJETO DE EXTENSAO\\n AUDIOVISUALIDADES  ELABORAR COM FOUCAULT  sob a coordenacao do\\nProfessor Doutor Nilton Milanez  do Departamento de Letras e Artes  desta Universidade.\\n\\nArtigo 2deg   Esta Resolugao entra em vigor na data de sua aprovagao \\n\\nrevogadas as disposigdes em contrario.\\n

In [302]:
scores

{5: {'score': 3.220498302961827},
 3: {'score': 2.8845247029357046},
 2: {'score': 2.8751778059606723},
 4: {'score': 2.8658912879485934},
 6: {'score': 2.8614744079848062}}

In [269]:
cursor = conn.cursor()  
cursor.execute("SELECT * FROM resolucoes")
rows = cursor.fetchall()

In [136]:
'''
  Table termos:
  refs @ https://github.com/amandassa/DBR-engenho-de-busca/tree/master
'''
for row in rows:
  print(row)

{'id': 334, 'term': ''}
{'id': 293, 'term': '0'}
{'id': 278, 'term': '001'}
{'id': 248, 'term': '01'}
{'id': 362, 'term': '02'}
{'id': 261, 'term': '04'}
{'id': 329, 'term': '07'}
{'id': 366, 'term': '074'}
{'id': 247, 'term': '08'}
{'id': 291, 'term': '1'}
{'id': 339, 'term': '11'}
{'id': 269, 'term': '12'}
{'id': 376, 'term': '120'}
{'id': 363, 'term': '131'}
{'id': 273, 'term': '14'}
{'id': 340, 'term': '142'}
{'id': 373, 'term': '15'}
{'id': 336, 'term': '17'}
{'id': 268, 'term': '19'}
{'id': 319, 'term': '2'}
{'id': 274, 'term': '2004'}
{'id': 249, 'term': '2016'}
{'id': 335, 'term': '2017'}
{'id': 368, 'term': '21'}
{'id': 337, 'term': '228'}
{'id': 338, 'term': '25'}
{'id': 260, 'term': '27'}
{'id': 272, 'term': '271'}
{'id': 259, 'term': '496'}
{'id': 262, 'term': '76'}
{'id': 258, 'term': '77'}
{'id': 267, 'term': '86'}
{'id': 266, 'term': '874'}
{'id': 271, 'term': '9'}
{'id': 316, 'term': 'Academy'}
{'id': 301, 'term': 'Amazonia'}
{'id': 324, 'term': 'aprovacao'}
{'id': 372,

In [138]:
'''
  Table documentos:
  refs @ https://github.com/amandassa/DBR-engenho-de-busca/tree/master
'''
for row in rows:
  print(row)

{'id': 1, 'id_resolucao': 2, 'id_termo': 243, 'frequencia': 2}
{'id': 2, 'id_resolucao': 2, 'id_termo': 244, 'frequencia': 1}
{'id': 3, 'id_resolucao': 2, 'id_termo': 245, 'frequencia': 1}
{'id': 4, 'id_resolucao': 2, 'id_termo': 246, 'frequencia': 1}
{'id': 5, 'id_resolucao': 2, 'id_termo': 247, 'frequencia': 1}
{'id': 6, 'id_resolucao': 2, 'id_termo': 248, 'frequencia': 1}
{'id': 7, 'id_resolucao': 2, 'id_termo': 249, 'frequencia': 3}
{'id': 8, 'id_resolucao': 2, 'id_termo': 250, 'frequencia': 1}
{'id': 9, 'id_resolucao': 2, 'id_termo': 251, 'frequencia': 1}
{'id': 10, 'id_resolucao': 2, 'id_termo': 252, 'frequencia': 1}
{'id': 11, 'id_resolucao': 2, 'id_termo': 253, 'frequencia': 1}
{'id': 12, 'id_resolucao': 2, 'id_termo': 254, 'frequencia': 1}
{'id': 13, 'id_resolucao': 2, 'id_termo': 255, 'frequencia': 2}
{'id': 14, 'id_resolucao': 2, 'id_termo': 256, 'frequencia': 1}
{'id': 15, 'id_resolucao': 2, 'id_termo': 257, 'frequencia': 2}
{'id': 16, 'id_resolucao': 2, 'id_termo': 258, 'f

In [270]:
'''
  Table resolucoes:
  refs @ https://github.com/amandassa/DBR-engenho-de-busca/tree/master
'''
for row in rows:
  print(row)
  print('----------------------------------------------------------------------------------------------------------------------')

{'id': 2, 'ano': '2016', 'data_insercao': '11/06/2023', 'data_publicacao': '07/01/2016', 'reitor': 'Evandro do Nasdimento Silva', 'texto': 'REITORIA/UEFS\nPUBLICADO D.O.E.\n\nEm,08 / 01 / 2016\n\n  \n\nUNIVERSIDADE ESTADUAL DE FEIRA DE SANTANA\nAutorizada pelo Decreto Federal N° 77.496 de 27/04/76\n\nReconhecida pela Portaria Ministerial N° 874/86 de 19/12/86\nRecredenciada pelo Decreto Estadual n° 9.271 de 14/12/2004\nGABINETE DA REITORIA\n\nRESOLUCAO CONSEPE 001/2016\n\nO Reitor da Universidade Estadual de Feira de Santana e Presidente do\nCONSELHO SUPERIOR DE ENSINO, PESQUISA E EXTENSAO, no uso de suas\natribui¢gdes,\n\nRESOLVE:\n\nArtigo 1° - Aprovar 0 PROJETO DE PESQUISA “Micota\nassociada a Hevea spp nativa da regiaio da Amazonia brasileira”, sob a\ncoordenacéo do Professor Doutor Aristételes Gées Neto, do Departamento de\n\nCiéncias Biologicas, desta Universidade, financiado pela National Academy of\nSciences.\n\nArtigo 2° - Esta Resolug&o entra em vigor na data de sua aprovacao